In [8]:
import os
import struct

# Salsa20 constants
ROUNDS = 20
SIGMA = b"expand 32-byte k"


def salsa20_core(key, nonce):
    # Initialize state
    state = [
                struct.unpack("<I", SIGMA[i * 4:i * 4 + 4])[0] for i in range(4)
            ] + [
                struct.unpack("<I", key[i * 4:i * 4 + 4])[0] for i in range(8)
            ] + [
                struct.unpack("<I", nonce[i * 4:i * 4 + 4])[0] for i in range(3)
            ] + [0]

    x = state[:]

    for _ in range(ROUNDS // 2):
        for i in range(0, 16, 4):
            x[i] += x[i + 1]
            x[i + 3] ^= x[i]
            x[i + 2] += x[i + 3]
            x[i + 1] ^= x[i + 2]

    return [(x[i] + state[i]) & 0xFFFFFFFF for i in range(16)]


def salsa20_encrypt(plaintext, key, nonce):
    keystream = bytearray()
    key_schedule = salsa20_core(key, nonce)

    for i in range(len(plaintext)):
        if i % 64 == 0:
            key_schedule = salsa20_core(key, nonce)
            nonce = (struct.unpack("<I", nonce[:4])[0] + 1).to_bytes(4, 'little') + nonce[4:]

        keystream.append(plaintext[i] ^ key_schedule[i % 64])

    return bytes(keystream)



In [10]:

def main():
    key = os.urandom(32)  # Random 32-byte key
    nonce = os.urandom(8)  # Random 8-byte nonce
    plaintext = b"Hello, this is a secret message!"

    ciphertext = salsa20_encrypt(plaintext, key, nonce)
    print("Ciphertext:", ciphertext)

    decrypted = salsa20_encrypt(ciphertext, key, nonce)
    print("Decrypted text:", decrypted.decode())


In [11]:
main()

error: unpack requires a buffer of 4 bytes